In [1]:
import multiprocessing
import pandas as pd
import os
import numpy as np
import json
import math
import threading
import gc
import time

In [2]:
datasets = os.path.join(os.getcwd(),'Datasets')

movies_path = os.path.join(datasets,'movies_dataset')

user_based_dict_path = os.path.join(movies_path,'utility_matrix_user_based.txt')
item_based_dict_path = os.path.join(movies_path,'utility_matrix_item_based.txt')
metadata_path = os.path.join(movies_path,'movies_metadata.txt')


CORES = multiprocessing.cpu_count()
cliques = {}

np.random.seed(10)

In [3]:
# load the metadata file, with topic informations ecc
with open(metadata_path,'r') as fr:
    movies_metadata = json.load(fr)


def convert_metadata_to_int(movies_metadata):
    movies_metadata_int = {}

    for key1 in movies_metadata.keys():
        key11 = int(key1)

        movies_metadata_int[key11] = movies_metadata[key1]
    
    return movies_metadata_int

movies_metadata = convert_metadata_to_int(movies_metadata)


In [19]:
# load movies datasets
with open(user_based_dict_path,'r') as fr:
    user_based_utility_matrix = json.load(fr)
'''
with open(item_based_dict_path,'r') as fr:
    item_based_utility_movies = json.load(fr)
'''

"\nwith open(item_based_dict_path,'r') as fr:\n    item_based_utility_movies = json.load(fr)\n"

In [20]:
def convert_ratings_to_float(user_based_utility):
    user_based_correct = {}

    for key1 in user_based_utility.keys():
        for key2 in user_based_utility[key1].keys():

            key11 = int(float(key1))
            key22 = int(float(key2))

            if(user_based_correct.get(key11) == None):
                user_based_correct[key11] = {}
            
            user_based_correct[key11][key22] = float(user_based_utility[key1][key2])
    
    return user_based_correct

user_based_utility_matrix = convert_ratings_to_float(user_based_utility_matrix)

gc.collect()

9821

In [21]:
removed = []

def remove_items_not_in_metadata(user_based_utility, movies_metadata):
    for key1 in user_based_utility.keys():
        dict_1 = user_based_utility[key1]
        keys = list(dict_1.keys()).copy()
        for idx in range(len(keys)):
            key2 = keys[idx]
            if movies_metadata.get(key2) == None:
                print('item {} has no metadata, eliminating'.format(key2))
                removed.append(key2)
                del dict_1[key2]
    return user_based_utility, removed

user_based_utility_matrix, removed = remove_items_not_in_metadata(user_based_utility_matrix, movies_metadata)
removed_unique = np.unique(removed)
removed_unique

1 has no metadata, eliminating
item 52281 has no metadata, eliminating
item 27611 has no metadata, eliminating
item 150548 has no metadata, eliminating
item 27611 has no metadata, eliminating
item 52281 has no metadata, eliminating
item 7669 has no metadata, eliminating
item 27611 has no metadata, eliminating
item 52281 has no metadata, eliminating
item 150548 has no metadata, eliminating
item 155968 has no metadata, eliminating
item 720 has no metadata, eliminating
item 52281 has no metadata, eliminating
item 720 has no metadata, eliminating
item 52281 has no metadata, eliminating
item 85780 has no metadata, eliminating
item 27611 has no metadata, eliminating
item 720 has no metadata, eliminating
item 62336 has no metadata, eliminating
item 7669 has no metadata, eliminating
item 62336 has no metadata, eliminating
item 26693 has no metadata, eliminating
item 4207 has no metadata, eliminating
item 720 has no metadata, eliminating
item 27611 has no metadata, eliminating
item 52281 has no

array([   142,    720,    730,    769,    770,    791,    821,   1107,
         1122,   1133,   1142,   1166,   1316,   1421,   1434,   1709,
         2258,   2270,   2851,   4051,   4207,   4568,   5069,   5209,
         5738,   6955,   7669,  26379,  26587,  26649,  26693,  27049,
        27611,  27708,  27724,  30991,  31193,  32352,  32600,  38198,
        40697,  42987,  52281,  55207,  62336,  69849,  70828,  73370,
        73759,  77328,  77330,  77359,  77854,  79299,  81731,  85780,
        87073,  87308,  90647,  90945,  93988,  96062,  96075,  99766,
       100044, 100058, 100089, 100450, 100538, 103277, 104155, 106334,
       106642, 107718, 107780, 108727, 108977, 118392, 119565, 122926,
       126106, 139195, 143351, 144050, 147250, 150548, 151763, 152284,
       155968, 163921, 167570, 169906, 170745, 171749, 173535])

In [22]:
# functions to operate division in test/train dictionaries

def divide_dataset(dataset):
    folds = [{},{},{},{}]

    for key1 in dataset.keys():
        for key2 in dataset[key1].keys():
            rand_index = np.random.randint(4)
            
            tmp_dict = folds[rand_index]

            if(tmp_dict.get(key1) == None):
                tmp_dict[key1] = {}
            
            tmp_dict[key1][key2] = dataset[key1][key2]
    
    return folds

def merge_dicts(dicts):
    ret = {}

    for i in range(len(dicts)):
        tmp_dict = dicts[i]
        for key1 in tmp_dict.keys():
            for key2 in tmp_dict[key1].keys():

                if(ret.get(key1)==None):
                    ret[key1] = {}
                ret[key1][key2] = tmp_dict[key1][key2]

    return ret


# divide the dataset in train/test set

folds = divide_dataset(user_based_utility_matrix)

train_dict = merge_dicts([fold for fold in folds[:3]])
test_dict = folds[3] 

#try to clear memory usage
del user_based_utility_matrix
del folds
gc.collect()

0

In [23]:
# function used in computing similarity measure between two users 


"""
    Calculates the average value of the 
    valeues in a

    a -> dictionary of non zero values 
"""
def avg(a):
    i=0
    tot = 0
    for k in a.keys():
        i +=1
        tot += a.get(k)
    
    return tot/i

"""
    Calculates a new dictionary that for each
    non null value of a has that values - const

    returns a - const

    a -> dictioary of non zero values
"""
def scale(a,const):
    ret = {}
    
    for k in a.keys():
        ret[k] = a.get(k) - const
    
    return ret

"""
    calculates the norm of a dictionary

    a -> dictioary of non zero values
"""
def norm (a):
    tot = 0
    for k in a.keys():
        tot += pow(a.get(k),2) 

    return math.sqrt(tot)

"""
    computes the inner product of two dictionaries

    a -> dictioary of non zero values
    b -> dictioary of non zero values

"""
def inner_product(a,b):
    tot = 0
    for k in a.keys():
        b_tmp = b.get(k)
        if(b_tmp != None):
            tot += b_tmp * a.get(k) 

    return tot

"""
    Calculates the correlation coefficent
        between two vectors
    
    a, b dictionaries of non zero values

    esplicit -> boolean value that  defines if we
    need to scale or not cause in the case of implicit
    we do not need to scale
"""
def compute_correlation_coefficent(a,b):

    avg_a = avg(a)
    avg_b = avg(b)

    a_scaled = scale(a, avg_a)
    b_scaled = scale(b, avg_b)

    a_scaled_norm = norm(a_scaled)
    b_scaled_norm = norm(b_scaled)

    if(a_scaled_norm == 0 or b_scaled_norm == 0):
        # print('One of the two vectors has 0 norm, returning 0.')
        return 0 

    sim = inner_product(a_scaled,b_scaled)/(a_scaled_norm*b_scaled_norm)

    return sim

def compute_similarities(user_ids, utility_matrix, user_dict):
    similarities = []
    for user in user_ids:
        tmp_user_dict = utility_matrix[user]
        similarity = compute_correlation_coefficent(user_dict, tmp_user_dict)
        similarities.append([user,similarity])
    
    return similarities


# definition of threads that will compute similarities for cliques generation
class ComputeCorrelationCoefficentThread (threading.Thread):
   """
      user_dict -> dictionary of the user: {itemID:rating}
      user_ids -> set of users IDs
      utility_matrix -> dataset
   """
   def __init__(self, user_dict, user_ids, utility_matrix, name = None):
      threading.Thread.__init__(self)
      self.name = name
      self.user_dict = user_dict
      self.user_ids = user_ids
      self.utility_matrix = utility_matrix
      self.result = None
   
   def run(self):
      if self.name == None:
         raise Exception('Something bad happened, thread has no name')
      #print('Thread {} started'.format(self.name))
      self.result = compute_similarities(self.user_ids, self.utility_matrix, self.user_dict)
      

   # vector of similarities --> [userID, sim_score]
   def join(self):
        threading.Thread.join(self)
        if self.result is not None:
            return self.result
        else:
            print('Error using threads, result is None')

In [24]:
# partition a list of unique items (data) into num_folds sublists

def make_partitions(num_folds, data):
    data_size = len(data)
    fold_size = data_size // num_folds
    partitions = []
    for idx in range(num_folds-1):
        partitions.append(data[idx*fold_size:(idx+1)*fold_size])
    partitions.append(data[idx*fold_size:])
    return partitions


"""
    Returns the clique of the user as np.array

    user -> userID
    utility_matrix -> {user:{item:rating}}
    clique_size -> size of the clique of the user
"""
def compute_clique(user, utility_matrix, clique_size):

    unique_users = list(set(utility_matrix.keys()))
    user_dict = utility_matrix[user]
    
    
    partitions = make_partitions(CORES,unique_users)        

    threads = []
    for idx in range(CORES):
        name = "Thread-{}".format(idx)
        thread = ComputeCorrelationCoefficentThread(name = name, 
                                                user_dict = user_dict, 
                                                user_ids = partitions[idx], 
                                                utility_matrix = utility_matrix)
        threads.append(thread)

    
    [thread.start() for thread in threads]    
    print('\n======= {} threads started ======='.format(CORES))

    results = [thread.join() for thread in threads]
    print('\n======= all threads joined =======')
    
    similarities = []
    for elem in results:
        for similarity in elem:
            similarities.append(similarity)
    
    '''
    print('Starting computing similarities with a single core')
    similarities = compute_similarities(unique_users, utility_matrix, user_dict)
    '''

    similarities = np.array(similarities)
    clique = similarities[np.argsort(similarities[:,1])[::-1]][1:clique_size+1]

    return clique

In [25]:
"""
    returns the predicted rating given a user and an item

    user -> userID
    item -> itemID
    clique -> list of users that are similar to userID
"""

def predict(user, item, clique, utility_matrix):
    numerator = 0
    denominator = 0
    for elem in clique:
        neighbor = elem[0]
        similarity = elem[1]

        neigh_dict = utility_matrix[neighbor]
        rating = neigh_dict.get(item)
        
        if  rating == None:
            continue

        numerator += rating*similarity
        denominator += similarity
    
    if denominator == 0:
        #print('denominator is 0, no user in the clique rated this item')
        return -1
    
    return numerator/denominator

In [26]:
# randomly select a subset of k users in test_dict for which we will score the recommender system 
# test users must have at least 15 ratings to test (so that, after removing the ones that we cannot predict (if no user in the clique rated the same item) we should have 10 ratings to rank)

def select_test_users(train_dict, test_dict, movies_metadata, test_size, min_ratings_per_user):
    np.random.seed(7)

    selected_test_users = []

    all_test_users = list(test_dict.keys())
    print('\nThe number of users in test set is {}'.format(len(all_test_users), flush=True))
    print('\nWe are taking into consideration only users with more than {} ratings'.format(min_ratings_per_user))
    print('\nThe subset of the test set taken into consideration is of size {}'.format(test_size))

    all_train_users = list(train_dict.keys())

    i=1
    while i<=test_size:
        #print('i is {}'.format(i), flush=True)
        idx = np.random.randint(len(all_test_users))

        user = all_test_users[idx]
        if user not in all_train_users:
            print("User is not in train set, can't compute clique")
            continue

        user_items = list(test_dict[user].keys())
        for item in user_items:
            if movies_metadata.get(item) == None:
                print("user has an item with no metadata, can't diversify with it")
                continue

        # check if user has at least 15 ratings
        num_user_ratings = len(list(test_dict[user].keys()))
        if num_user_ratings < min_ratings_per_user:
            #print('user has {} ratings'.format(num_user_ratings), flush=True)
            continue      

        # check if we already considered this user
        if user in selected_test_users:
            #print('user already considered', flush=True)
            continue

        # add user to list of selected users
        selected_test_users.append(user)
        i+=1

    return selected_test_users
    


In [27]:
# list value is computed by taking the true rating of each element returned multyplied with a factor that depends on the position at which the element is in the ranked recommendation predicted

def compute_list_value(predicted_ranking, true_ranking):
    list_value = 0
    for index, row in predicted_ranking.iterrows():
        rating = true_ranking[true_ranking['item'] == row['item']]['true_rating'].values[0]
        item_score = (1/(np.log(index+np.e)))*rating 
        list_value += item_score

        # print(index, rating)
    return list_value
    

In [28]:
# definition of jacccard coefficient score and dissimilarity measure

def jaccard_score(list_1, list_2):
    intersection = []
    union = []
    for elem in list_1:
        if elem not in intersection and elem in list_2:
            intersection.append(elem)
        union.append(elem)
    for elem in list_2:
        if elem not in union:
            union.append(elem)

    score = len(intersection)/len(union)
    return score

def dissimilarity_measure(id_1, id_2, movies_metadata):
    metadata_1 = movies_metadata[id_1]
    metadata_2 = movies_metadata[id_2]

    #spoken_language_1 = metadata_1['spoken_language']
    #spoken_language_2 = metadata_2['spoken_language']

    original_language_1 = [metadata_1['original_language']]
    original_language_2 = [metadata_2['original_language']]

    production_companies_1 = metadata_1['production_companies']
    production_companies_2 = metadata_2['production_companies']

    production_countries_1 = metadata_1['production_countries']
    production_countries_2 = metadata_2['production_countries']

    genres_1 = metadata_1['genres']
    genres_2 = metadata_2['genres']

    #spoken_language_score = jaccard_score(spoken_language_1, spoken_language_2)
    original_language_score = jaccard_score(original_language_1, original_language_2)
    production_companies_score = jaccard_score(production_companies_1, production_companies_2)
    production_countries_score = jaccard_score(production_countries_1, production_countries_2)
    genres_score = jaccard_score(genres_1, genres_2)    

    scores = np.array([original_language_score, production_companies_score, production_countries_score, genres_score])

    weights = np.array([0.15, 0.15, 0.1, 0.6])

    similarity = np.dot(weights, scores)
    return 1-similarity

In [29]:
def intra_list_similarity(l,movies_metadata):
    similarity = 0
    n_items = len(l)

    for i in range(n_items - 1):
        for j in range(i + 1, n_items):
            (item1, item2) = (l[i], l[j])
            similarity += 1-dissimilarity_measure(item1, item2, movies_metadata)

    return similarity

In [30]:
def differentiation_algorithm(old_list, movies_metadata,diversification_factor):
    new_list = old_list[:1] # The first element remains always the same
    old_ranking = [list((rank, old_list[rank - 1], 0)) for rank in range(1, len(old_list) + 1)] # List of triples (rank(starting from 1), item, dissimilarity_actual_value)
    new_item = new_list[0]
    

    for i in range(1, 10):
        old_ranking = list(filter(lambda x: x[1] != new_item, old_ranking)) # Remove the element passed in the previous iteration to the new list from the old ranking
        dissimilarities = []
        
        for index in range(len(old_ranking)):
            triple = old_ranking[index]
            old_ranking[index][2] += dissimilarity_measure(int(triple[1]), int(new_item), movies_metadata) # Update the dissimilarity value considering the last added item (avg)
            dissimilarities.append(old_ranking[index])
        
        dissimilarities.sort(reverse=True, key=lambda tup: tup[2]) # Sort in decreasing order according to the dissimilarity value
        dissimilarity_rank = 0
        min_rank = len(old_list) + 1 + len(dissimilarities)
        
        for triple in dissimilarities:
            new_rank = triple[0] * (1 - diversification_factor) + dissimilarity_rank * diversification_factor # We consider the rank for both the old list and the dissimilarity sorted list
            if new_rank < min_rank:
                new_item = triple[1]
                min_rank = new_rank
            dissimilarity_rank += 1
        
        new_list.append(new_item)
    
    return new_list

In [34]:
# select optimal ranking, predicted ranking, and optimal top 10, then score the result using list_value metric
def score_predictions(predictions, movies_metadata, diversification_factor):
    optimal_ranking = predictions[['item', 'true_rating']].sort_values(by=['true_rating'], ascending=False).reset_index(drop=True)

    predicted = predictions[['item', 'predicted_rating']].sort_values(by=['predicted_rating'], ascending=False).reset_index(drop=True)

    if diversification_factor != 0:
        # print('diversification factor != 0, start differentiating')
        predicted_top_10 = differentiation_algorithm(list(predicted['item'].values), movies_metadata, diversification_factor)
        mask = [elem in predicted_top_10 for elem in predicted['item'].values]
        predicted_top_10 = predicted[mask].reset_index(drop=True)
    else:
        predicted_top_10 = predicted[:10]
    optimal_top_10 = optimal_ranking[:10]

    # compute similarity score of the original list and the diversified list
    sim_original = intra_list_similarity(list(predicted['item'][:10].values), movies_metadata)
    sim_diversified = intra_list_similarity(list(predicted_top_10['item'].values), movies_metadata)

    # compute list value both for the optimal ranking and the predicted one
    list_value = compute_list_value(predicted_top_10, optimal_ranking)
    best_list_value = compute_list_value(optimal_top_10, optimal_ranking)
    
    # the metric is MAE, considering as error the difference of the list values of the optimal top 10 list and the predicted one
    difference = best_list_value - list_value
    print('Optimal ranking value: {}\tpredicted ranking value: {}\t difference: {}'.format(best_list_value, list_value,difference))

    return difference, sim_original, sim_diversified

In [31]:
def perform_test(train_dict, test_dict, movies_metadata, clique_size = 50, test_size = 5, min_ratings_per_user = 20, differentiation_factor = 0):

    print('\nClique size is {}'.format(clique_size))

    selected_test_users = select_test_users(train_dict, test_dict, movies_metadata, test_size, min_ratings_per_user)
    assert(len(selected_test_users) == test_size, "Sizes don't match: something is wrong with selecting test users!")

    MAE = 0
    mean_original_similarity = 0
    mean_diversified_similarity = 0
    for user in selected_test_users:
        
        test_items = list(test_dict[user].keys())
        clique = compute_clique(user, train_dict, clique_size)


        predictions_and_truth = pd.DataFrame( np.array( [[item, predict(user, item, clique, train_dict), test_dict[user][item]] for item in test_items] ),columns=['item','predicted_rating','true_rating'] )
        predictions_and_truth = predictions_and_truth[predictions_and_truth.predicted_rating != -1]

        difference, sim_original, sim_diversified = score_predictions(predictions_and_truth, movies_metadata, differentiation_factor)
        assert(difference >= 0, 'Error: list value greater than best list value')
        
        MAE += difference
        mean_original_similarity += sim_original
        mean_diversified_similarity += sim_diversified
        
        print('\nintra-list similarity score of original list is {}, while the diversified list scored {}'.format(sim_original, sim_diversified))
        

    MAE = MAE/test_size
    mean_original_similarity = mean_original_similarity/test_size
    mean_diversified_similarity = mean_diversified_similarity/test_size
    
    print('\n======= TEST TERMINATED ======= ')
    print('Mean absolute error is {}\nMean intra-similarity score for non-diversified predictions is {}\nMean intra-similarity score for diversified predictions is {}\nResults have been computed on a subset of the test dictionary of {} users'.format(MAE, mean_original_similarity, mean_diversified_similarity, test_size ))

    return MAE, mean_original_similarity, mean_diversified_similarity

In [32]:
diversification_factors = list(np.linspace(0,1,21))
test_size = 100
min_ratings_per_user = 30

MAEs = {}
mean_similarities_non_diversified = {}
mean_similarities_diversified = {}
for diversification_factor in diversification_factors:
    start = time.time()
    
    MAE, mean_similarity_non_diversified, mean_similarity_diversified = perform_test(train_dict, 
                                                                                test_dict,  
                                                                                movies_metadata, 
                                                                                clique_size=200, 
                                                                                test_size = test_size, 
                                                                                min_ratings_per_user=min_ratings_per_user,
                                                                                differentiation_factor = diversification_factor)
    MAEs[diversification_factor] = MAE
    mean_similarities_non_diversified[diversification_factor] = mean_similarity_non_diversified
    mean_similarities_diversified[diversification_factor] = mean_similarity_diversified
    
    end = time.time()
    print('Elapsed time is {} seconds'.format(end-start))


Clique size is 200

The number of users in test set is 232351

We are taking into consideration only users with more than 30 ratings

The subset of the test set taken into consideration is of size 100

======= 12 threads started =======


KeyboardInterrupt: 

In [ ]:
print('\n======= MEAN ABSOLUTE ERRORS =======')
print(MAEs)

print('\n======= MEAN SIMILARITIES NON DIVERSIFIED =======')
print(mean_similarities_non_diversified)

print('\n======= MEAN SIMILARITIES DIVERSIFIED =======')
print(mean_similarities_diversified)